In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate, compile_kwargs
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-16 12:15:18.358515: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 12:15:18.966782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [7]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_min1_to_1)

In [4]:
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)

2023-10-16 12:15:29.016894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 12:15:29.043206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 12:15:29.043296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 12:15:29.044899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 12:15:29.044963: I tensorflow/compile

In [20]:
model = SAT1TopologicalConv(5, 8, len(data.samples), len(data.labels))
model.compile(**compile_kwargs)

In [17]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)

In [5]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20


2023-10-16 12:15:38.127548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-16 12:15:39.175511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-16 12:15:39.181077: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f576b0c5b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-16 12:15:39.181102: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-16 12:15:39.184628: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-16 12:15:39.279693: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

755/755 [==============================] - 62s 75ms/step - loss: 0.8215 - accuracy: 0.6748 - val_loss: 0.5097 - val_accuracy: 0.7990
Epoch 2/20
755/755 [==============================] - 56s 74ms/step - loss: 0.5508 - accuracy: 0.7922 - val_loss: 0.4366 - val_accuracy: 0.8273
Epoch 3/20
755/755 [==============================] - 56s 73ms/step - loss: 0.4947 - accuracy: 0.8136 - val_loss: 0.4084 - val_accuracy: 0.8375
Epoch 4/20
755/755 [==============================] - 57s 75ms/step - loss: 0.4661 - accuracy: 0.8235 - val_loss: 0.4000 - val_accuracy: 0.8401
Epoch 5/20
755/755 [==============================] - 56s 74ms/step - loss: 0.4361 - accuracy: 0.8316 - val_loss: 0.4118 - val_accuracy: 0.8329
Epoch 6/20
755/755 [==============================] - 56s 74ms/step - loss: 0.4164 - accuracy: 0.8379 - val_loss: 0.4030 - val_accuracy: 0.8411
Epoch 7/20
755/755 [==============================] - 57s 75ms/step - loss: 0.4133 - accuracy: 0.8420 - val_loss: 0.4090 - val_accuracy: 0.8431
Epo

(<keras.src.callbacks.History at 0x7f1a33c3da90>,
 {'confirmation': {'precision': 0.5967302452316077,
   'recall': 0.49772727272727274,
   'f1-score': 0.5427509293680297,
   'support': 440},
  'decision': {'precision': 0.7303128371089536,
   'recall': 0.7754868270332188,
   'f1-score': 0.7522222222222221,
   'support': 873},
  'encoding': {'precision': 0.8783638320775027,
   'recall': 0.9325714285714286,
   'f1-score': 0.9046563192904656,
   'support': 875},
  'pre-attentive': {'precision': 0.9277389277389277,
   'recall': 0.9353701527614571,
   'f1-score': 0.9315389116442365,
   'support': 851},
  'response': {'precision': 0.9880810488676997,
   'recall': 0.9409761634506243,
   'f1-score': 0.963953488372093,
   'support': 881},
  'accuracy': 0.8512755102040817,
  'macro avg': {'precision': 0.8242453782049383,
   'recall': 0.8164263689088003,
   'f1-score': 0.8190243741794093,
   'support': 3920},
  'weighted avg': {'precision': 0.8491576528888034,
   'recall': 0.8512755102040817,
   '

In [ ]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

In [6]:
model.save("models/cnn_base_bigger_conv")

INFO:tensorflow:Assets written to: models/cnn_base_bigger_conv/assets


INFO:tensorflow:Assets written to: models/cnn_base_bigger_conv/assets


In [16]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 770, 30, 1)]      0         
                                                                 
 masking (Masking)           (None, 770, 30, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 746, 30, 64)       1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 373, 30, 64)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 357, 30, 128)      139392    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 178, 30, 128)      0         
 g2D)                                                        